In [7]:
pip install fastembed

Note: you may need to restart the kernel to use updated packages.


In [8]:
from fastembed import TextEmbedding

In [9]:
model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [10]:
embeddings = list(model.embed(["I just discovered the course. Can I join now?"]))

In [11]:
# Q1. Embedding the query
query_vector = embeddings[0]
print("Size:", query_vector.shape)
print("Min value:", query_vector.min())

Size: (512,)
Min value: -0.11726373551188797


In [12]:
import numpy as np

In [13]:
np.linalg.norm(query_vector)

1.0000000000000002

In [14]:
query_vector.dot(query_vector)

1.0000000000000004

In [15]:
# Q2. Cosine similarity with another vector
doc = 'Can I still join the course after the start date?'
doc_embed = list(model.embed([doc]))
query_vector.dot(doc_embed[0])

0.9008528856818033

In [16]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [17]:
# Q3. Ranking by cosine
for doc in documents:
    text_embed = list(model.embed([doc['text']]))
    print(text_embed[0].dot(query_vector))

0.7629684493123694
0.8182378325403554
0.808539729076283
0.7133078600724644
0.7304499388234829


In [18]:
# Q4. Ranking by cosine, version two
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    ft = list(model.embed([full_text]))[0]
    print(ft.dot(query_vector))

0.85145431212252
0.8436594058703557
0.8408287206920109
0.775515771189969
0.8086008033213792


Is it different from Q3?
Yes. 

In [20]:
models = TextEmbedding.list_supported_models()

In [21]:
# Q5. Selecting the embedding model
mods = []
for mod in models:
    mods.append([mod['dim'], mod['model']])
print(min(mods))

[384, 'BAAI/bge-small-en']


In [22]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
model2 = TextEmbedding(model_name='BAAI/bge-small-en')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [40]:
# Q6. Indexing with qdrant (2 points)
dots = []
for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    t = list(model.embed([text]))[0]
    dots.append(t.dot(query_vector))
max(dots)

0.8620737814476882